In [1]:
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [2]:
df =  pd.read_csv('books.csv',sep=";",on_bad_lines="skip",encoding='latin-1')
titles = df['Book-Title']

/var/folders/fn/j00_0ypj1cjd3q6s6m3_sptw0000gn/T/ipykernel_342/1624946487.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('books.csv',sep=";",on_bad_lines="skip",encoding='latin-1')


In [12]:
chars = sorted(list(set("".join(titles[i] for i in range(len(titles))))))
chars.insert(0,"ç")
stoi = {char:i for i,char in enumerate(chars)}
itos = {i:char for i,char in enumerate(chars)}

print(stoi)

{'ç': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '=': 29, '>': 30, '?': 31, '@': 32, 'A': 33, 'B': 34, 'C': 35, 'D': 36, 'E': 37, 'F': 38, 'G': 39, 'H': 40, 'I': 41, 'J': 42, 'K': 43, 'L': 44, 'M': 45, 'N': 46, 'O': 47, 'P': 48, 'Q': 49, 'R': 50, 'S': 51, 'T': 52, 'U': 53, 'V': 54, 'W': 55, 'X': 56, 'Y': 57, 'Z': 58, '[': 59, '\\': 60, ']': 61, '^': 62, '_': 63, '`': 64, 'a': 65, 'b': 66, 'c': 67, 'd': 68, 'e': 69, 'f': 70, 'g': 71, 'h': 72, 'i': 73, 'j': 74, 'k': 75, 'l': 76, 'm': 77, 'n': 78, 'o': 79, 'p': 80, 'q': 81, 'r': 82, 's': 83, 't': 84, 'u': 85, 'v': 86, 'w': 87, 'x': 88, 'y': 89, 'z': 90, '{': 91, '|': 92, '}': 93, '~': 94, '\xa0': 95, '¡': 96, '¢': 97, '£': 98, '¤': 99, '¥': 100, '¦': 101, '§': 102, '¨': 103, '©': 104, 'ª': 105, '«': 106, '¬': 107, '\xad': 108, '®': 109, '¯

In [15]:
xs = []
ys = []
for title in titles:
    title = "ç" + title + "ç"
    for char1,char2 in zip(title,title[1:]):
        xs.append(stoi[char1])
        ys.append(stoi[char2])

In [78]:
N = torch.ones((len(chars),len(chars)))
N.shape

torch.Size([127, 127])

In [81]:
for ix,iy in zip(xs,xs[1:]):
    N[ix,iy] += 1
    

In [82]:
p = N / N.sum(1,keepdim=True)


In [13]:
ix = 0
res = ""
while True:
    ix = torch.multinomial(p[ix],num_samples=1).item()
    if ix == 0:
        break
    res += itos[ix]

print(res)

NameError: name 'p' is not defined

In [96]:
log_likely = 0
n = 0


for ix,iy in zip(xs,xs[1:]):
    prob = p[ix,iy]
    log = torch.log(prob)
    log_likely += log
    n += 1

print(-log_likely/n)

tensor(2.5878)


In [16]:
W = torch.randn((len(chars),len(chars)),requires_grad=True)

In [17]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

number of examples:  10537226


In [31]:
W.grad = None

In [188]:
for _ in range(500):
    x = F.one_hot(torch.tensor(xs[:105372]),num_classes=len(chars)).float()
    h = x @ W
    h = h.exp()
    probs = h / h.sum(1,keepdim=True)
    loss = -probs[torch.arange(105372),ys[:105372]].log().mean() + 0.01*(W**2).mean()

    print(loss)

    loss.backward()
    W.data = -50 * W.grad

/var/folders/fn/j00_0ypj1cjd3q6s6m3_sptw0000gn/T/ipykernel_99492/4228874764.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = F.one_hot(torch.tensor(xs[:105372]),num_classes=len(chars)).float()


tensor(2.6270, grad_fn=<AddBackward0>)
tensor(2.6268, grad_fn=<AddBackward0>)
tensor(2.6267, grad_fn=<AddBackward0>)
tensor(2.6265, grad_fn=<AddBackward0>)
tensor(2.6264, grad_fn=<AddBackward0>)
tensor(2.6262, grad_fn=<AddBackward0>)
tensor(2.6260, grad_fn=<AddBackward0>)
tensor(2.6259, grad_fn=<AddBackward0>)
tensor(2.6257, grad_fn=<AddBackward0>)
tensor(2.6256, grad_fn=<AddBackward0>)
tensor(2.6254, grad_fn=<AddBackward0>)
tensor(2.6253, grad_fn=<AddBackward0>)
tensor(2.6251, grad_fn=<AddBackward0>)
tensor(2.6250, grad_fn=<AddBackward0>)
tensor(2.6248, grad_fn=<AddBackward0>)
tensor(2.6247, grad_fn=<AddBackward0>)
tensor(2.6245, grad_fn=<AddBackward0>)
tensor(2.6244, grad_fn=<AddBackward0>)
tensor(2.6242, grad_fn=<AddBackward0>)
tensor(2.6241, grad_fn=<AddBackward0>)
tensor(2.6239, grad_fn=<AddBackward0>)
tensor(2.6238, grad_fn=<AddBackward0>)
tensor(2.6236, grad_fn=<AddBackward0>)
tensor(2.6235, grad_fn=<AddBackward0>)
tensor(2.6234, grad_fn=<AddBackward0>)
tensor(2.6232, grad_fn=<A

In [203]:
ix = 0
res = ""
while True:
    sample = F.one_hot(torch.tensor([ix]),num_classes=len(chars)).float()
    sample = sample @ W
    sample = sample.exp()
    sample = sample / sample.sum()
    ix = torch.multinomial(sample,num_samples=1).item()
    #ix = torch.multinomial(p[ix],num_samples=1).item()
    if ix == 0:
        break
    res += itos[ix]

print(res)

Pok Ma Therburine Bond Droreilof Grne Deb Ag Morisig SEand Ale
